<a href="https://colab.research.google.com/github/MohammedShabry/DL-Assignment/blob/shabry/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load dataset
df = pd.read_csv('Telco-Customer-Churn.csv')

# Drop unnecessary columns
df = df.drop(columns=['customerID'])

# Handle missing values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill missing values only for numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Convert categorical features into numeric using Label Encoding
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df['Partner'] = le.fit_transform(df['Partner'])
df['Dependents'] = le.fit_transform(df['Dependents'])
df['PhoneService'] = le.fit_transform(df['PhoneService'])
df['MultipleLines'] = le.fit_transform(df['MultipleLines'])
df['InternetService'] = le.fit_transform(df['InternetService'])
df['OnlineSecurity'] = le.fit_transform(df['OnlineSecurity'])
df['OnlineBackup'] = le.fit_transform(df['OnlineBackup'])
df['DeviceProtection'] = le.fit_transform(df['DeviceProtection'])
df['TechSupport'] = le.fit_transform(df['TechSupport'])
df['StreamingTV'] = le.fit_transform(df['StreamingTV'])
df['StreamingMovies'] = le.fit_transform(df['StreamingMovies'])
df['Contract'] = le.fit_transform(df['Contract'])
df['PaperlessBilling'] = le.fit_transform(df['PaperlessBilling'])
df['PaymentMethod'] = le.fit_transform(df['PaymentMethod'])
df['Churn'] = le.fit_transform(df['Churn'])

# Select input features and target
X = df.drop(columns=['Churn'])
y = df['Churn']

# Normalize the numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [16]:
# Reshape data to 3D format for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=64, activation='tanh', input_shape=(X_train_reshaped.shape[1], 1)))
lstm_model.add(Dense(units=32, activation='relu'))
lstm_model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm_history = lstm_model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate on the test set
lstm_score = lstm_model.evaluate(X_test_reshaped, y_test)
print(f'LSTM Test Accuracy: {lstm_score[1]:.4f}')

Epoch 1/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.7097 - loss: 0.5938 - val_accuracy: 0.7533 - val_loss: 0.4944
Epoch 2/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7320 - loss: 0.5106 - val_accuracy: 0.7657 - val_loss: 0.4830
Epoch 3/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7474 - loss: 0.4930 - val_accuracy: 0.7764 - val_loss: 0.4791
Epoch 4/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7474 - loss: 0.4986 - val_accuracy: 0.7782 - val_loss: 0.4566
Epoch 5/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.7573 - loss: 0.4819 - val_accuracy: 0.7995 - val_loss: 0.4292
Epoch 6/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7686 - loss: 0.4767 - val_accuracy: 0.8004 - val_loss: 0.4207
Epoch 7/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7789 - loss: 0.4581 - val_accuracy: 0.7897 - val_loss: 0.4192
Epoch 8/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7926 - loss: 0.4457 - 

In [17]:
# Evaluate on test data
lstm_preds = lstm_model.predict(X_test_reshaped)
lstm_preds = (lstm_preds > 0.5).astype(int)

# Performance evaluation
from sklearn.metrics import accuracy_score, classification_report

# Classification report (Precision, Recall, F1-Score)
print(classification_report(y_test, lstm_preds))

# Accuracy score
accuracy = accuracy_score(y_test, lstm_preds)
print(f"LSTM Model Accuracy: {accuracy * 100:.2f}%")


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1036
           1       0.58      0.54      0.56       373

    accuracy                           0.77      1409
   macro avg       0.71      0.70      0.70      1409
weighted avg       0.77      0.77      0.77      1409

LSTM Model Accuracy: 77.43%
